In [7]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

def create_user_profile(dict_rating, df_sim):
    df = pd.DataFrame(index=[0], columns=df_sim.columns)
    for key, value in dict_rating.items():
        df[key] = value
    
    df = df.T.fillna(0)
    df.rename(columns={0: 'rating'}, inplace=True)

    return df

def recommend_games(df_new_user, df_sim):
    # Convert df_similarity and df_new_user to numpy arrays
    df_similarity_array = df_sim.to_numpy()
    df_new_user_array = df_new_user.to_numpy()

    # Calculate the dot product (weighted ratings)
    weighted_ratings = np.dot(df_similarity_array, df_new_user_array)

    # Normalize the ratings by dividing by the sum of similarities for each item
    sum_of_similarities = df_sim.sum(axis=1)
    normalized_ratings = weighted_ratings / sum_of_similarities[:, np.newaxis]

    # Min-max scaling to a 0-10 range
    min_score = np.min(normalized_ratings)
    max_score = np.max(normalized_ratings)
    normalized_ratings = 10 * (normalized_ratings - min_score) / (max_score - min_score)

    # Convert the result to a DataFrame
    recommended_games_df = pd.DataFrame(normalized_ratings, index=df_sim.index, columns=["score"])

    # Sort the recommended games in descending order
    top_recommendations = recommended_games_df.loc[df_new_user.index].sort_values(by="score", ascending=False)

    # Return the top recommendations as a DataFrame
    return top_recommendations

def get_top_recommendations(dict_rating, df_sim, num_recommendations=10):
    df = pd.DataFrame(list(dict_rating.items()), columns=['name', 'rating'])
    mean_value = df['rating'].mean()

    dict_rating = dict(zip(df['name'], df['rating']))
    df_new_user = create_user_profile(dict_rating, df_sim)

    condition = (df_new_user['rating'] < mean_value) & (df_new_user['rating'] != 0) & (df_new_user['rating'] < 5)
    df_new_user.loc[condition, 'rating'] -= mean_value

    df_recommeds = recommend_games(df_new_user, df_sim)
    rec_games = df_recommeds.loc[df_new_user[df_new_user.values == 0].index].sort_values(by="score", ascending=False)[:num_recommendations]

    return rec_games

In [8]:
file_path = 'https://storage.googleapis.com/boardgame-recommu/files/boardgames_sim.csv'

# Read the CSV file using dask
df_sim = dd.read_csv(file_path)
df_sim

Dask DataFrame Structure:
                 name ...and then, we held hands. ...und tschüss! 10 Days in Africa 10 Days in Asia 10 Days in Europe 10 Days in the Americas 10 Days in the USA 10 Minute Heist: The Wizard's Tower 10' to Kill 1000 Blank White Cards 1066, Tears to Many Mothers 11 nimmt!  12 Days 12 Thieves 13 Clues 13 Days: The Cuban Missile Crisis 13 Minutes: The Cuban Missile Crisis, 1962 1655: Habemus Papam 1714: The Case of the Catalans 1754: Conquest – The French and Indian War 1775: Rebellion 1805: Sea of Glory 1812: The Invasion of Canada     1817 1822: The Railways of Great Britain 1844/1854 1846: The Race for the Midwest 1849: The Game of Sicilian Railways 1856: Railroading in Upper Canada from 1856 1860: Railways on the Isle of Wight 1861: The Railways of the Russian Empire 1862: Railway Mania in the Eastern Counties 1880: China 1882: Assiniboia     18AL     18CZ 18Chesapeake     18EU 18Lilliput    18MEX 1906 San Francisco 1911 Amundsen vs Scott 1920 Wall Street 1936: Guerra Civil 1942 USS Yorktown 1944: Race to the Rhine 1960: The Making of the President     1969 1987 Channel Tunnel 1989: Dawn of Freedom 1st & Goal 2 de Mayo 2038: Tycoons of the Asteroid Belt 20th Century       23 25 Words or Less 2GM Tactics 30 Rails 30 Seconds   4 Gods 5-Minute Dungeon 5-Minute Marvel 5-Minute Mystery 50 Clues: The Pendulum of the Dead      504 51st State 51st State: Master Set     5211 6 nimmt! 25 Jahre   7 Ages 7 Days of Westerplatte  7 Ronin 7 Wonders 7 Wonders (Second Edition) 7 Wonders Duel 7 Wonders: Architects 7th Sea Collectible Card Game 878 Vikings: Invasions of England 8Bit Box A Brief History of the World A Castle for All Seasons A Column of Fire A Distant Plain A Fake Artist Goes to New York A Feast for Odin A Few Acres of Snow A Fistful of Meeples A Fistful of Penguins A Game of Thrones A Game of Thrones Collectible Card Game A Game of Thrones: Catan – Brotherhood of the Watch A Game of Thrones: Hand of the King A Game of Thrones: The Board Game (Second Edition) A Game of Thrones: The Card Game A Game of Thrones: The Card Game (Second Edition) A Gentle Rain A Handful of Stars A Most Dangerous Time: Japan in Chaos, 1570-1584 A Song of Ice & Fire: Tabletop Miniatures Game – Stark vs Lannister Starter Set A Study in Emerald A Study in Emerald (Second Edition) A Tale of Pirates A Thief's Fortune A Touch of Evil: Dark Gothic A Touch of Evil: The Supernatural Game A Victory Denied: Crisis at Smolensk, July-September, 1941 A Victory Lost: Crisis in Ukraine 1942-1943 A War of Whispers A World at War A4 Quest ALIEN: Fate of the Nostromo AT-43 Initiation Set: Operation Damocles Abandon All Artichokes Abandon Ship Abomination: The Heir of Frankenstein Above and Below Abracada...What?    Abyss Ad Astra Ada's Library Adios Amigos Adrenaline Advanced Squad Leader: Starter Kit #1 Advanced Squad Leader: Starter Kit #2 Advanced Squad Leader: Starter Kit #3 Advanced Squad Leader: Starter Kit Expansion Pack #1 Adventure Games: The Dungeon Adventure Land Adventure Tactics: Domianne's Tower Adventure Time Card Wars: Finn vs. Jake Adventure Time Fluxx Aeon's End Aeon's End: Legacy Aeon's End: Outcasts Aeon's End: The New Age Aeon's End: War Eternal   Aerion Aeroplanes: Aviation Ascendant   Africa Africana Afrika: 2nd Edition After The Empire After The Virus After the Flood Aftermath Agamemnon Age of Civilization Age of Conan: The Strategy Board Game Age of Discovery Age of Empires III: The Age of Discovery Age of Gods Age of Industry Age of Mythology: The Boardgame Age of Napoleon Age of Renaissance Age of Steam Age of Thieves Age of Towers Age of War Agent Decker Agents of SMERSH  Agility     Agra Agricola Agricola (Revised Edition) Agricola: All Creatures Big and Small Agricola: Family Edition Agropolis Air Baron Air, Land & Sea Airlines Europe Airship City Airships Akrotiri Al Cabohne Al Rashid Aladdin's Dragons Alcatraz: The Scapegoat Alchemist Alchemists Alea Iacta Est Alexandros Alhambra Alhambra: The Dice Game Alice's Garde

In [9]:
df_sim = pd.read_csv('https://storage.googleapis.com/boardgame-recommu/files/boardgames_sim.csv')
df_sim

KeyboardInterrupt: 